## EEG data grouing

In [82]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경로 설정
eeg_folder_path = '/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/EEG_train_csv'

# 파일 목록 가져오기
file_list = os.listdir(eeg_folder_path)

# 그룹별로 파일들을 그룹화
grouped_files = {}
for file_name in file_list:
    group_key = file_name[:11]
    if group_key == ".DS_Store":  # .DS_Store 그룹은 건너뜁니다
        continue
    if group_key not in grouped_files:
        grouped_files[group_key] = []
    grouped_files[group_key].append(file_name)

In [83]:
# 새로운 데이터를 저장할 리스트
new_data = []

# 그룹별로 데이터 처리
for key, file_names in grouped_files.items():
    # 그룹 내 파일들을 읽어와 데이터 리스트에 추가
    group_data = []
    for file_name in file_names:
        file_path = os.path.join(eeg_folder_path, file_name)
        df = pd.read_csv(file_path, encoding='latin1')  # 인코딩 변경
        channel_data = df.iloc[:, 0].values  # 첫 번째 열의 데이터만 사용
        group_data.append(channel_data)
    
    # 그룹 데이터를 평균하여 새로운 데이터 생성
    new_group_data = np.mean(group_data, axis=0)
    new_data.append(new_group_data)

# 저장할 폴더 경로
output_folder_path = '/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/EEG_train2_csv'

# 데이터를 CSV 파일로 저장
for i, group_data in enumerate(new_data):
    file_name = f"group_{i+1}.csv"
    file_path = os.path.join(output_folder_path, file_name)
    pd.DataFrame(group_data).to_csv(file_path, index=False)

In [ ]:
# 그룹 개수 출력
print(f"그룹 개수: {len(grouped_files)}")

# 그룹 이름 출력 (정렬하여)
for group_name in sorted(grouped_files.keys()):
    print(group_name)

# EEG preprocessing

In [84]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경로 설정
eeg_folder_path = '/Users/sh_oh/Library/CloudStorage/Dropbox/Data/2023-1/BDP/ECSMP_Dataset/EEG_train2_csv'

# 파일 목록 가져오기
file_list = os.listdir(eeg_folder_path)

# 전체 데이터를 저장할 리스트
data = []

# 가장 긴 데이터 길이를 기준으로 zero-padding
max_length = 0

# 데이터 읽어오기 및 전처리
for file_name in file_list:
    file_path = os.path.join(eeg_folder_path, file_name)
    df = pd.read_csv(file_path)
    channel_data = df.iloc[:, 0].values  # 첫 번째 열의 데이터만 사용
    data.append(channel_data)
    if len(channel_data) > max_length:
        max_length = len(channel_data)

# Zero-padding
padded_data = pad_sequences(data, maxlen=max_length, padding='post')

# 데이터 정규화
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(padded_data)

# Denoising AutoEncoder(DAE) model

In [89]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.optimizers.legacy import Adam

# 가우시안 노이즈 추가
def add_gaussian_noise(data, noise_factor):
    noise = np.random.normal(loc=0.0, scale=noise_factor, size=data.shape)
    noisy_data = data + noise
    return noisy_data

In [90]:
# Denoising Autoencoder 모델 생성
def create_denoising_autoencoder(input_shape, encoding_dim):
    input_layer = Input(shape=input_shape)
    
    # Encoder
    encoded = Flatten()(input_layer)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    # Decoder
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(encoded)
    decoded = Reshape(input_shape)(decoded)

    # Autoencoder
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return autoencoder

In [92]:
# 입력 데이터의 형태
input_shape = normalized_data.shape[1:]
encoding_dim = 64

# 가우시안 노이즈 추가
noise_factor = 0.5
noisy_data = add_gaussian_noise(normalized_data, noise_factor)

# Denoising Autoencoder 모델 생성
autoencoder = create_denoising_autoencoder(input_shape, encoding_dim)

# 모델 학습
autoencoder.fit(noisy_data, normalized_data, epochs=50, batch_size=6, shuffle=True, verbose=1)

Epoch 1/50
39/39 [==============================] - 1s 11ms/step - loss: 0.0668
Epoch 2/50
39/39 [==============================] - 1s 14ms/step - loss: 0.0423
Epoch 3/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0355
Epoch 4/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0339
Epoch 5/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0307
Epoch 6/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0292
Epoch 7/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0283
Epoch 8/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0276
Epoch 9/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0269
Epoch 10/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0264
Epoch 11/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0264
Epoch 12/50
39/39 [==============================] - 1s 15ms/step - loss: 0.0259
Epoch 13/50
39/39 [==================

ValueError: No such layer: dense. Existing layers are: ['input_8', 'flatten_6', 'dense_8', 'dense_9', 'reshape_2'].

# feature extraction

In [94]:
# 특징 추출 (인코더의 출력)
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_8').output)
features = encoder.predict(normalized_data)

8/8 [==============================] - 0s 7ms/step


In [95]:
print(features.dtype)

float32
